In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T

import numpy as np
import math
import random
import gym
import matplotlib
import matplotlib.pyplot as plt

from collections import namedtuple
from itertools import count
from PIL import Image

In [3]:
env = gym.make('CartPole-v0').unwrapped

# Setup matplotlib.
is_python = 'inline' in matplotlib.get_backend()
if is_python:
    from IPython import display

plt.ion()

# If GPU is to be used.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

## Reply Memory

We’ll be using experience replay memory for training our DQN. It stores the transitions that the agent observes, allowing
us to reuse this data later. By sampling from it randomly, the transitions that build up a batch are decorrelated. It has
been shown that this greatly stabilizes and improves the DQN training procedure.
